<a href="https://colab.research.google.com/github/kjapan87/movie_conversation_2020_12_06/blob/main/movie_conversation_2020_12_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
import re 
import time

In [ ]:
lines = open('movie_lines.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')
conversations = open('movie_conversations.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')

FileNotFoundError: ignored

In [ ]:
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

#in id2line just having line pai
conversations_ids = []
for conversation in conversations[:-1]:
    _conversation = conversation.split(' +++$+++ ')[-1][1:-1].replace("'", "").replace(" ", "")
    conversations_ids.append(_conversation.split(','))

In [ ]:
questions = []
answers = []
for conversation in conversations_ids:
  for i in range(len(conversation) - 1):
    questions.append(id2line[conversation[i]])
    answers.append(id2line[conversation[i+1]])  

In [ ]:
#preprocessing function definition
def cleanText(text):
    text = text.lower()
    text = re.sub("[^a-zA-Z]"," ",text)
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"we'd", "we would", text)    
    text = re.sub(r"[-()\"#/@;:<>{}+=~|.?,]", "", text)
    return text
    
#preprocessing the "questions"
clean_questions = []
for question in questions:
    clean_questions.append(cleanText(question))

#preprocessing the "answers"
clean_answers = []
for answer in answers:
    clean_answers.append(cleanText(answer))    


In [ ]:
#create a dictionary that map each word to its no. of occurences
#we can use many different kind of models , Bag of words, TDIDF, Word2Vec, BERT etc.
#however these models are on lexicological level, but we have sequential data, so we created this for loop.
word2count = {}
for question in clean_questions:
    for word in question.split():
        if word not in word2count:
          word2count[word] = 1
        else:
          word2count[word] += 1

for answer in clean_answers:
    for word in answer.split():
        if word not in word2count:
          word2count[word] = 1
        else:
          word2count[word] += 1    


In [ ]:
##filtering (aprox.) 5% of extra-words & sorting the occurences to remove repeated entries
threshold_ques = 20
word_number = 0

question2int_dict = {}
for word,count in word2count.items():
    if count >= threshold_ques:
        question2int_dict[word] = word_number
        word_number += 1
        
threshold_ans = 20
answers2int_dict = {}
for word,count in word2count.items():
    if count >= threshold_ans:
        answers2int_dict[word] = word_number
        word_number += 1    

In [ ]:
# Adding 4 tags <EOS> (end of sentence) & <SOS> (start of sentence), <padding>, <out> 
#<OUT> token was created to replace less-frequent word occurence.
    
tokens = ["<PAD>","<EOS>","<OUT>","<SOS>"]

for token in tokens:
    question2int_dict[token] = len(question2int_dict)+1
    
for token in tokens:
    answers2int_dict[token] = len(answers2int_dict)+1

# Inversing a dictionary key:values --> values:key
answers2int_wordDict = {word_i:word for word,word_i in answers2int_dict.items()}    


In [ ]:

#Adding string token to end of every Clean_answers
# those 4 tags added above was at word-level and now we again add the tags to sentence-level.
#because decoder needs to understand the starting of decoding process (as identifiers)

for i in range(len(clean_answers)):
    clean_answers[i] += " <EOS>"


In [ ]:
# translating all the questions &answers into integer 
#using the dictionary that was created and replacing the words to occurence in integers.
# & replacing all the words that were filtered out by <OUT> tag.


out_questions = []
#now the words we removed (5% less frequent word occurrence), how to accomodate these
#for these <OUT> words, which were removed, and now again looking 
#at sentence level, we need the <OUT> tag to replace for those missing part. 
for question in clean_questions:
  integ=[] #using full-sentences & using the created dictionarie
  for word in question.split():
    if word not in question2int_dict:
      integ.append(question2int_dict['<OUT>'])
    else:
      integ.append(question2int_dict[word])
  out_questions.append(integ)

out_answers = []

for answer in clean_answers:
  integ=[] #using full-sentences & using the created dictionaries
  for word in answer.split():
    if word not in answers2int_dict:
      integ.append(answers2int_dict['<OUT>'])
    else:
      integ.append(answers2int_dict[word])
  out_answers.append(integ)


In [ ]:
#sort all the questions & answers
#sort the sentences in question2int which have all the questions as integer
#for training puropse, so we sorted the clean out_question as ascending order.
# starting from 1 word sentence, to many word sentence.
#fetch the 

sorted_out_question = []
sorted_out_answers = []
for length in range(1, 10+1):
    for i in enumerate(out_questions):
        if len(i[1]) == length:
            sorted_out_question.append(out_questions[i[0]])
            sorted_out_answers.append(out_answers[i[0]])


In [ ]:
#sequences to sequence model

##create placeholder (tensors)
##input of 4 raw material for building the archtitecture of the model.
##input, target, learningrate,keepProb.
##in tensorflow all the variables are passed as placeholder(variables in tensorflow)
##placeholders are very advanced array.
##advanced data-structure

def model_input():
    inputs = tf.compat.v1.placeholder(tf.int32,[None,None],name = "input")
    target = tf.compat.v1.placeholder(tf.int32,[None,None],name = "output")
    learning_rate = tf.compat.v1.placeholder(tf.float32,name = "learning_rate")
    keep_prob = tf.compat.v1.placeholder(tf.float32,name = "dropout")
    return inputs,target,learning_rate,keep_prob    

In [ ]:
#Preprocessing for Decoder as it will predict the answers.
##decoder requirement : we require <SOS> in the begining of the sorted_answers
##decoder requirement : we require all the sorted_answers in batches

def preprocess (target,target2int,batch_size):
    left_side = tf.fill([batch_size,1], target2int["<SOS>"], name=None)
    ##tf.fill is function is filing an empty array and filling with scalar values
    ##tf.fill (left side) will add <SOS>
    ##tf.fill (right side) will have all the answers except the last entry (which is <EOS>)
    ##Get 2 seperate columns, in the end we join both of these Left & right side together.
    right_side = tf.strided_slice(input = target, begin = [0,0], end = [batch_size,-1], strides=[1,1], begin_mask=0, end_mask=0, ellipsis_mask=0,new_axis_mask=0, shrink_axis_mask=0, var=None, name=None)
    concatenate_2_side =tf.concat(values = [left_side,right_side], axis = 1)
    return concatenate_2_side
    

In [ ]:
#ENCODER
##Encoder won't have training or testing data difference.
##will work in a similar way for both training/testing.
##batches of questions are going to be fed in the encoder
##Encoder"s basic function is to understand the fed text.

def encoder (rnn_input,rnn_size,rnn_layers,keep_prob,sequence_len):
    create_LSTM = tf.contrib.rnn.LSTMCell(rnn_size)
    LSTM_dropout = tf.contrib.rnn.DropoutWrapper(create_LSTM,input_keep_prob = keep_prob)
    encoder_cell = tf.contrib.rnn.MultiRNNCell([LSTM_dropout]*rnn_layers)
    _,encoder_state = tf.nn.bidirectional_dynamic_rnn(cell_forward = encoder_cell, cell_backwards = encoder_cell,sequence_length = sequence_len,inputs = rnn_input,dtype = tf.float32)
    return encoder_state #only encoder state will go to the decoder
    ##input_keep_prob is a variable to keep track of keep_prob
    ##ecoder is going to be stacked layers
    ##bidirectional RNN returns 2 parameters, 1st is encoder_states, 2nd not used
    ##
    
    
    #tf.contrib is used to create LSTM cells
##rnn_input is the input to the RNN.
##rnn_size =  how many number of input tensors of the the encoder.
##rnn_layers = no. of layers
##keep_prob is to apply dropout regularization, which is used to improve to
##control drop-out rate(neurons which we choose to overwrite) so that we can 
##activate, (usually kept at 20% during training iterations).
##sometime dropout rates fluctuates (too high, or too low) so keep_prob will help to control this fluctuation.
##sequence_len is the list of length of each question in a batch.

In [ ]:
#DECODER FOR TRAINING DATA
def decoder_training_data (encoder_state, decoder_cell, decoder_embedding_input, sequence_len, decoding_scope, output_decoder, keep_prob , batch_size):
    attention_states = tf.zeros([batch_size, 1, decoder_cell.output_size], dtype=tf.dtypes.float32, name=None) #initiallization of weights of attention state
    attention_keys, attention_values, attention_score_fn, attention_construct_fn = tf.contrib.seq2seq.prepare_attention(attention_states, attention_option = "Bahdanau", num_units = decoder_cell.output_size, reuse=False) #Adding attention features & prepare attention module
    ##attention_keys = the keys to be compared to te target state.
    ##attention_values = is the value to construct the context vectors. Context is returned by encoder & used by the deceoder.
    ##attention_score = is used to compute the similarity between key & target states
    ##attention_construct = is used to build the attention states.
    ##attention_column is the column of the dataset.
    ##decoding_scope will wrap all the variables(contextual_vector, with attention mechanism)
    ##Attention are weights, so (not integers but matrix)
    ## because there are multiple hidden states, 
    ##and each hidden layer has a weight attached, so they need to be stored in the form of matrix)
    ##attention state, has 3 dimensions, 
    ##row = batch size, column= 1, elements = output of decoder cell of hidden layer
    training_decoder_fn = tf.contrib.seq2seq.attention_decoder_fn_train(encoder_state[0], attention_keys, attention_values, attention_score_fn, attention_construct_fn, name="AttentionDecoderTraining") #pass all features of attention declared in the previous line of code.
    decoder_output,_,_ = tf.contrib.seq2seq.dynamic_rnn_decoder(decoder_cell, training_decoder_fn, decoder_embedding_input, sequence_len, decoding_scope, name="DecoderOutputforTraining")
    decoder_dropout = tf.nn.dropout(decoder_output, keep_prob, name="DecoderDropoutController")
    return decoder_output(decoder_dropout)#helping to updates the weights as it will retrain with feedback mechanism.


In [ ]:
#DECODER FOR TESTING DATA

##Decoder function of the testing is going to predict the output of the test set.
##We will have a Validation set, which will also be given to Training data.
##But also Cross-Validation will be done, which is a technique where we keep small part of ou data 
##to test the predictive power of of model's observation. It is useful to remove overfitting, & 
##also improves the accuracy on the new observations of the new data.
def decoder_testing_data (encoder_state, decoder_cell, decoder_embedding_input, start_of_sequence_id, end_of_sequence_id, maximum_length, num_decoder_symbols, sequence_len, decoding_scope, output_fn, keep_prob , batch_size):
    attention_states = tf.zeros([batch_size, 1, decoder_cell.output_size], dtype=tf.dtypes.float32, name=None) #initiallization of weights of attention state
    attention_keys, attention_values, attention_score_fn, attention_construct_fn = tf.contrib.seq2seq.prepare_attention(attention_states, attention_option = "Bahdanau", num_units = decoder_cell.output_size, reuse=False) #Adding attention features & prepare attention module
    testing_decoder_fn = tf.contrib.seq2seq.attention_decoder_fn_inference(output_fn, encoder_state[0], attention_keys, attention_values, attention_score_fn, attention_construct_fn, decoder_embedding_input, start_of_sequence_id, end_of_sequence_id, maximum_length, num_decoder_symbols, dtype=tf.int32, name=None) #maximum_length is the length of longest answer it can find in the batch.#num_decoder_symbol is the total numbers of words in the answers, we need to the answers2int_wordDict.
    ##once the chatbot is trained, a logic inside its brain exist, & therefore, able to deduce logically
    ##the answers that is being asked. So  it makes its own logic during the training phases
    ##and during the inference phase, it will use this logic and use it during testing
    test_predictions,_,_ = tf.contrib.seq2seq.dynamic_rnn_decoder(decoder_cell, testing_decoder_fn, decoding_scope, name="DecoderOutputforTesting")
    return test_predictions

In [ ]:
def decoder (decoder_embedding_input,decoder_embedding_matrix, encoder_state, num_decoder_symbols, rnn_size, rnn_layers, target2int, keep_prob, sequence_len, batch_size):
    with tf.variable_scope("decoding") as decoding_scope:
        create_LSTM = tf.contrib.rnn.LSTMCell(rnn_size)
        LSTM_dropout = tf.contrib.rnn.DropoutWrapper(create_LSTM,input_keep_prob = keep_prob)
        decoder_cell = tf.contrib.rnn.MultiRNNCell([LSTM_dropout]*rnn_layers)##we need to initialize some weights, that will be associated with the neurons of fully-connected layer(LAST LAYER of RNN)
        weights = tf.truncated_normal_initializer(mean=0.0, stddev=1.0, seed=None, dtype=tf.dtypes.float32)
        bias = tf.zeros_initializer()
        output_fully_connected_layer_fn = lambda x: tf.contrib.layers.fully_connected(x,num_decoder_symbols,activation_fn=tf.nn.relu, normalizer_fn=None,normalizer_params=None,weights_initializer= weights,weights_regularizer=None,biases_initializer=bias,biases_regularizer=None,reuse=None,variables_collections=None,outputs_collections=None,trainable=True,scope=decoding_scope)
        ##lambda x was used because x can be user defined (text entry in chatbot)
        
        training_predictions = decoder_training_data (encoder_state, decoder_cell, decoder_embedding_input, sequence_len, decoding_scope, output_decoder, keep_prob , batch_size)
        decoding_scope.reuse_variables()
        testing_predictions = decoder_testing_data (encoder_state, decoder_cell, decoder_embedding_input, start_of_sequence_id, end_of_sequence_id, maximum_length, num_decoder_symbols, sequence_len-1, decoding_scope, output_fn, keep_prob , batch_size)
        
        return training_predictions,testing_predictions


In [ ]:
##Combining the Encoder & Decoder (Sequence to Sequence Model)
def seq2seq (inputs,target,keep_prob,sequence_len, batch_size, num_words_ques,num_words_anw,encoder_emb_size,decoder_emb_size,rnn_size,rnn_layers,quest_word2int):
    ## ASSEMBLAGE ## 
    encoder_embedded_input =tf.contrib.layers.embed_sequence(inputs,vocab_size=num_words_anw+1,embed_dim=encoder_emb_size,unique=False,initializer=tf.random_uniform_initializer(minval=0, maxval=1, seed=None),regularizer=None,trainable=True,scope=None,reuse=None)#tensorflow will automatically build
    encoder_state = encoder(encoder_embedded_input, rnn_size,rnn_layers,keep_prob,sequence_len)
    concatenate_2_side = preprocess (target,quest_word2int,batch_size)
    decoder_embedding_matrix = tf.Variable(tf.random.uniform([num_words_ques+1,decoder_emb_size], minval=0, maxval=1, dtype=tf.dtypes.float32, seed=None, name=None))
    decoder_embedding_input = tf.nn.embedding_lookup(decoder_embedding_matrix, concatenate_2_side, max_norm=None, name=None)
    training_predictions,testing_predictions = decoder (decoder_embedding_input,decoder_embedding_matrix, encoder_state, num_words_ques, rnn_size, rnn_layers, quest_word2int, keep_prob, sequence_len, batch_size)
    return training_predictions,testing_predictions




In [ ]:


#TRAINING THE SEQ2SEQ MODEL

##1) Setting the hyperparameter

epoch = 10
batch_size = 64
rnn_size = 512 ##no. of neurons in 1 RNN cell
rnn_layers = 3
encoder_emb_size = 512
decoder_emb_size = 512
learning_rate = 0.01 ##if this learning is to be too high then model is going to learn to fast, which isn't optimum (not too high or too low), the amount by which the weights will be updated aka step-size.(usually 0.0 to 1.0 ) 
learning_rate_decay = 0.90 ## learning_rate is going to be reduced after iteration of the training, so that the model can be learn deeply the logic.
min_learning_rate = 0.0001
keep_prob = 0.50 #recommended by geoffery hinton

In [ ]:
##2) defining a session, create an object for an interactive session class
    ##which is going to open interactive session for interactive flow
    ##we also need to reset the tensorflow graph to ensure, that the graph is ready for training 
    ##graph = in tensorflow a graph is made for all the calling function, and the flow of the calling (return values from 1 block to another)
tf.compat.v1.disable_eager_execution()
tf.compat.v1.reset_default_graph()
session = tf.compat.v1.InteractiveSession()

##3) taking model input
    
inputs,target,learning_rate,keep_prob = model_input()

##4) setting the sequence length

sequence_len = tf.compat.v1.placeholder_with_default(input = 25 , shape = None, name='sequencelength')

##5) Defining the Input Tensors

tf.shape(inputs, out_type=tf.dtypes.int32, name=None)


##6) Getting the training & test predictions
#training_predictions,testing_predictions = seq2seq (decoder_embedding_input,decoder_embedding_matrix, encoder_state, num_words_ques, rnn_size, rnn_layers, quest_word2int, keep_prob, sequence_len, batch_size)

#training_predictions,testing_predictions=seq2seq (inputs,target,keep_prob,sequence_len, batch_size, num_words_ques,num_words_anw,encoder_emb_size,decoder_emb_size,rnn_size,rnn_layers,quest_word2int)
 

